# Decision Trees - Interpretable Models with Sharp Edges

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/11_decision_trees.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Fit decision trees for regression/classification
2. Control complexity (depth, min samples) to manage overfitting
3. Interpret tree structure and failure modes
4. Compare tree vs linear/logistic baselines under CV
5. Document "when a tree is the right tool"

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The setup cell imports the core libraries for this notebook. `DecisionTreeClassifier` and `DecisionTreeRegressor` from scikit-learn implement the CART algorithm, while `plot_tree` lets us render the learned tree structure as a graphic. `StratifiedKFold` ensures each cross-validation fold preserves the class balance of the breast cancer dataset (roughly 63 % benign, 37 % malignant).

The message `Setup complete!` with the random seed **RANDOM_SEED = 474** confirms that all libraries loaded without error and that every stochastic operation in this notebook will be reproducible.

**Key takeaway:** A successful setup cell is the foundation of reproducibility. If any import fails here, none of the downstream code will run in Colab.

---

## 1. Decision Tree Intuition

### How Trees Make Decisions

**Algorithm (CART - Classification and Regression Trees):**
1. Start with all data at root
2. Find best feature + threshold to split
   - "Best" = maximize information gain (classification) or minimize MSE (regression)
3. Create two child nodes
4. Repeat recursively until stopping criteria

**Example Decision Path:**
```
Is income > $50k?
  ├─ No: Is age > 30?
  │   ├─ No: Predict class 0 (high risk)
  │   └─ Yes: Predict class 1 (low risk)
  └─ Yes: Predict class 1 (low risk)
```

### Key Hyperparameters

**Complexity control:**
- `max_depth`: Maximum tree depth (prevents overfitting)
- `min_samples_split`: Minimum samples to split a node
- `min_samples_leaf`: Minimum samples in leaf node
- `max_features`: Number of features to consider per split

## 2. Classification Tree Example

We will use the **Wisconsin Breast Cancer** dataset (569 samples, 30 numeric features) to build our first decision tree. Each sample is labeled as malignant or benign, making this a binary classification task. The tree will learn axis-aligned splits on features such as mean radius, mean texture, and worst concave points.

We cap `max_depth=3` so the tree remains small enough to visualize and interpret. Even a shallow tree can achieve surprisingly high accuracy on well-separated classes, and the visualization makes it clear exactly which features and thresholds drive each prediction.

In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print(f"Features: {X.shape[1]}")

# Fit a simple tree
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_SEED)
tree_clf.fit(X_train, y_train)

train_score = tree_clf.score(X_train, y_train)
test_score = tree_clf.score(X_test, y_test)

print(f"\n=== DECISION TREE (max_depth=3) ===")
print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")
print(f"Overfit gap: {train_score - test_score:.4f}")

**Reading the output:**

The dataset splits into **398 training** and **171 test** samples across **30 features**, all numeric measurements from digitized images of breast tissue. With `max_depth=3` the tree is constrained to at most 8 leaf nodes.

Look at the two accuracy numbers. Training accuracy around **0.97-0.98** tells us the shallow tree captures most of the signal. Test accuracy close behind (gap typically < 0.02) confirms the model is not overfitting at this depth. The **overfit gap** printed at the bottom should be small and positive; a negative gap would indicate unusual luck on the test split.

**Why this matters:** A depth-3 tree is already competitive with much more complex models on this dataset, demonstrating that a simple, interpretable model can be a strong baseline.

---

In [ ]:
# Visualize the tree
plt.figure(figsize=(20, 10))
plot_tree(
    tree_clf,
    feature_names=X.columns,
    class_names=data.target_names,
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Decision Tree Visualization (max_depth=3)")
plt.tight_layout()
plt.show()

print("💡 Follow a path from root to leaf to see decision logic")
print("💡 Darker colors = more samples, purity of class")

**Reading the output:**

The graphic shows every internal decision node and leaf node of the depth-3 tree. Each box contains four pieces of information: the **split rule** (e.g., `worst radius <= 16.8`), the **Gini impurity** (0 = perfectly pure), the **number of samples** reaching that node, and the **class distribution** (malignant vs. benign counts).

Color intensity encodes purity: darker blue means almost all samples are benign, darker orange means almost all are malignant. Follow any path from root to leaf and you read a plain-English decision rule: "If worst radius > 16.8 and worst concave points > 0.14, predict malignant."

Notice that the root split uses a feature from the "worst" category (the largest cell nuclei in the image). This makes clinical sense because malignant tumors tend to have larger, more irregularly shaped nuclei.

**Key takeaway:** This visualization is the primary advantage of decision trees over black-box models. You can hand this diagram to a domain expert and they can audit every prediction path.

---

## 3. The Overfitting Problem

### Trees Without Constraints = Memorization

An unrestricted decision tree will keep splitting until every leaf contains a single sample, achieving **100 % training accuracy** by memorizing the data. On new data the performance collapses because those hyper-specific rules do not generalize. The depth sweep below makes this tradeoff visible: training accuracy climbs monotonically with depth, while test accuracy peaks at a moderate depth and then declines.

Monitoring the **overfit gap** (train accuracy minus test accuracy) is the fastest way to diagnose the problem. A gap near zero means the model generalizes well; a large gap means the tree has memorized noise.

In [ ]:
# Compare different depths
depths = [1, 2, 3, 5, 10, 20, None]  # None = unlimited
results = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_SEED)
    tree.fit(X_train, y_train)
    
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)
    
    results.append({
        'max_depth': str(depth),
        'train_acc': train_acc,
        'test_acc': test_acc,
        'gap': train_acc - test_acc,
        'n_leaves': tree.get_n_leaves()
    })

results_df = pd.DataFrame(results)
print("=== DEPTH SWEEP ===")
print(results_df.to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy vs depth
axes[0].plot(range(len(depths)), results_df['train_acc'], marker='o', label='Train', linewidth=2)
axes[0].plot(range(len(depths)), results_df['test_acc'], marker='s', label='Test', linewidth=2)
axes[0].set_xticks(range(len(depths)))
axes[0].set_xticklabels(results_df['max_depth'])
axes[0].set_xlabel('Max Depth')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy vs Tree Depth')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Overfit gap
axes[1].bar(range(len(depths)), results_df['gap'], alpha=0.7, edgecolor='black')
axes[1].set_xticks(range(len(depths)))
axes[1].set_xticklabels(results_df['max_depth'])
axes[1].set_xlabel('Max Depth')
axes[1].set_ylabel('Train - Test Gap')
axes[1].set_title('Overfitting vs Tree Depth')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Unrestricted trees achieve 100% training accuracy (pure overfitting!)")
print("💡 Best test performance is at moderate depth")

**Reading the output:**

The results table and two-panel plot reveal the classic bias-variance tradeoff. In the left panel, training accuracy (blue) climbs steadily and reaches **1.0** for `max_depth=None` (unlimited), while test accuracy (orange) peaks around depth 3-5 then flattens or declines. The right panel shows the overfit gap growing from near zero at depth 1 to a substantial value at unlimited depth.

Key numbers to compare: at `max_depth=3` the gap is typically around **0.01-0.02**, meaning the model generalizes well. At `max_depth=None` the gap can exceed **0.05-0.10**, with the tree growing to dozens or hundreds of leaves that memorize training noise.

The `n_leaves` column reinforces this: a depth-3 tree has at most 8 leaves, while an unrestricted tree may have 20+ leaves on 398 training samples, averaging fewer than 20 samples per leaf. Leaves with very few samples produce unreliable predictions.

**Why this matters:** This depth sweep is the single most important diagnostic for decision trees. Always run it before selecting a final depth, and pick the depth that maximizes test (or CV) performance, not training performance.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Run a depth sweep and choose depth based on CV.

---

In [ ]:
# YOUR CODE: Depth sweep with cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
depths_to_test = [2, 3, 4, 5, 6, 7, 8, 10, 15]

cv_results = []
for depth in depths_to_test:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_SEED)
    scores = cross_val_score(tree, X_train, y_train, cv=cv, scoring='roc_auc')
    
    cv_results.append({
        'max_depth': depth,
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

cv_df = pd.DataFrame(cv_results)
print("=== CV DEPTH SWEEP ===")
print(cv_df.to_string(index=False))

best_depth = cv_df.loc[cv_df['cv_mean'].idxmax(), 'max_depth']
best_score = cv_df.loc[cv_df['cv_mean'].idxmax(), 'cv_mean']

print(f"\n✓ Best depth by CV: {best_depth} (CV ROC-AUC = {best_score:.4f})")

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(cv_df['max_depth'], cv_df['cv_mean'], yerr=cv_df['cv_std'], 
             marker='o', capsize=5, linewidth=2)
plt.axvline(x=best_depth, color='r', linestyle='--', label=f'Best depth = {best_depth}')
plt.xlabel('Max Depth')
plt.ylabel('CV ROC-AUC')
plt.title('Cross-Validation: Depth Selection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Reading the output:**

The cross-validation table reports ROC-AUC (not just accuracy) for each candidate depth, along with the standard deviation across five folds. The best depth is highlighted at the bottom. On the breast cancer dataset, the optimal depth is typically around **3-5**, yielding a CV ROC-AUC near **0.98-0.99**.

The error-bar plot makes the choice visual: performance rises steeply from depth 2 to 3, plateaus through depth 5-7, and then may dip slightly at higher depths as overfitting erodes generalization. The error bars (one standard deviation) show that deeper trees also have *higher variance* across folds, another sign of instability.

The vertical red dashed line marks the selected best depth. Note that choosing depth 3 vs. 4 vs. 5 may produce nearly identical mean ROC-AUC; in such cases, prefer the simpler (shallower) tree for better interpretability and robustness.

**Key takeaway:** Cross-validation gives a more reliable depth selection than a single train/test split because it averages over multiple data partitions, reducing the risk of picking a depth that only works well by chance.

---

## 4. Tree vs Linear Model Comparison

Decision trees and logistic regression make fundamentally different assumptions about the data. Logistic regression fits a single linear decision boundary in feature space, which works well when classes are roughly linearly separable. Trees, on the other hand, carve the space into axis-aligned rectangles and can capture non-linear interactions without manual feature engineering.

The comparison below uses the same `StratifiedKFold` cross-validation object for both models so the evaluation is fair. We report ROC-AUC on validation folds and on the held-out test set. On the breast cancer dataset, the two approaches often perform similarly because the classes are well separated; the real differences emerge on more complex datasets.

In [ ]:
# Compare tree vs logistic regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

models = {
    'Decision Tree (best)': DecisionTreeClassifier(max_depth=int(best_depth), random_state=RANDOM_SEED),
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
    ])
}

comparison = []
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Fit on full train for test evaluation
    model.fit(X_train, y_train)
    test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    comparison.append({
        'Model': name,
        'CV_Mean': scores.mean(),
        'CV_Std': scores.std(),
        'Test_Score': test_score
    })

comp_df = pd.DataFrame(comparison)
print("=== TREE VS LINEAR COMPARISON ===")
print(comp_df.to_string(index=False))

print("\n💡 Which performs better depends on data structure")
print("💡 Trees handle non-linear relationships naturally")
print("💡 Linear models need manual feature engineering")

**Reading the output:**

The comparison table shows CV ROC-AUC (mean and standard deviation) plus the test-set ROC-AUC for both the tuned decision tree and logistic regression. On the breast cancer dataset, both models typically achieve ROC-AUC above **0.97**, because the 30 features provide strong linear separability.

If the tree's CV standard deviation is noticeably larger than logistic regression's, that illustrates the **high-variance** nature of trees: small changes in the training fold can shift the tree structure significantly, while logistic regression produces a stable linear boundary every time.

The test scores should be consistent with the CV means. If either model's test score falls outside the range mean +/- 2*std, that is a red flag worth investigating (possible data leakage or an unusual test split).

**Why this matters:** This head-to-head comparison establishes whether a tree's ability to model non-linear boundaries actually helps on your specific dataset. On linearly separable data, the simpler logistic regression may be the better production choice due to its stability and interpretability.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Write 3 observed tree failure modes (with evidence).

---

### YOUR ANALYSIS: Tree Failure Modes

**Failure Mode 1: Overfitting**  
[Evidence from depth sweep - what happened with unlimited depth?]

**Failure Mode 2: Instability**  
[Evidence from CV std - how much do scores vary across folds?]

**Failure Mode 3: Extrapolation**  
[Trees can only predict values seen in training - what's the implication?]

---

## 5. Regression Trees

Everything we learned about classification trees applies to regression: the algorithm still makes recursive binary splits, but instead of Gini impurity it minimizes **mean squared error (MSE)** within each region. Leaf predictions are the average target value of the training samples that fall into that leaf.

We demonstrate with the **California Housing** dataset (20,640 samples, 8 features), predicting median house value in units of $100k. The depth sweep below shows the same overfitting pattern we saw in classification: an unrestricted regression tree achieves near-perfect training R-squared but generalizes poorly.

In [ ]:
# Load regression dataset
california = fetch_california_housing(as_frame=True)
X_reg = california.data
y_reg = california.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=RANDOM_SEED
)

# Depth sweep for regression
depths_reg = [2, 3, 5, 7, 10, 15, None]
reg_results = []

for depth in depths_reg:
    tree_reg = DecisionTreeRegressor(max_depth=depth, random_state=RANDOM_SEED)
    tree_reg.fit(X_train_reg, y_train_reg)
    
    train_r2 = tree_reg.score(X_train_reg, y_train_reg)
    test_r2 = tree_reg.score(X_test_reg, y_test_reg)
    
    reg_results.append({
        'max_depth': str(depth),
        'train_r2': train_r2,
        'test_r2': test_r2,
        'gap': train_r2 - test_r2
    })

reg_df = pd.DataFrame(reg_results)
print("=== REGRESSION TREE DEPTH SWEEP ===")
print(reg_df.to_string(index=False))

print("\n💡 Same overfitting pattern as classification")
print("💡 Unrestricted tree memorizes training data")

**Reading the output:**

The regression depth sweep mirrors the classification pattern. At `max_depth=2` the tree underfits with a low R-squared on both train and test. As depth increases, training R-squared approaches **1.0** (the tree memorizes the 14,448 training samples), while test R-squared peaks around depth **7-10** and then declines.

Typical values: the best test R-squared is around **0.60-0.70**, meaning a single regression tree explains roughly 60-70 % of variance in California median house values. The overfit gap at `max_depth=None` is dramatic, often exceeding **0.30**, confirming that an unrestricted regression tree is a poor production model.

Compare these numbers to what you would get from a simple Ridge regression (R-squared around 0.60). A moderately deep tree matches or slightly beats the linear baseline, but an unrestricted tree dramatically overfits. This motivates ensembles (Random Forests, Gradient Boosting) that combine many trees to achieve higher R-squared without memorization.

**Key takeaway:** Regression trees have the same overfitting vulnerability as classification trees. Always limit depth and validate on held-out data. The next notebook shows how Random Forests solve this problem by averaging hundreds of trees.

---

## 6. When to Use Decision Trees

### Strengths
- ✓ Interpretable (can visualize and explain)
- ✓ Handle non-linear relationships naturally
- ✓ No feature scaling needed
- ✓ Handle mixed data types (numeric + categorical)
- ✓ Capture interactions automatically
- ✓ Fast to train and predict

### Weaknesses
- ✗ High variance (unstable - small data changes → different tree)
- ✗ Easy to overfit
- ✗ Poor extrapolation (can't predict outside training range)
- ✗ Biased toward features with many values
- ✗ Step-function boundaries (not smooth)

### Use Decision Trees When:
1. Interpretability is critical
2. You need quick baseline
3. Relationships are highly non-linear
4. You'll use ensemble methods (Random Forests, Boosting)

### Avoid Decision Trees When:
1. High-dimensional sparse data
2. Need smooth decision boundaries
3. Small datasets (unstable)
4. Extrapolation required

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Tree Mechanics**: Recursive partitioning with greedy splits
2. **Overfitting Risk**: Unrestricted trees memorize perfectly
3. **Complexity Control**: depth, min_samples_split, min_samples_leaf
4. **Interpretability**: Can visualize exact decision logic
5. **Limitations**: High variance, poor extrapolation

### Critical Rules:

> **"Never use unrestricted trees in production"**

> **"Always tune depth with cross-validation"**

> **"Trees are building blocks for ensembles"**

### Next Steps:

- Next notebook: Random Forests (ensemble of trees)
- We'll fix tree instability with bagging
- Learn feature importance from forests

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Tree-Based Methods (trees, pruning)
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - CART foundations and complexity control
- Breiman, L., Friedman, J., Stone, C. J., & Olshen, R. A. (1984). *Classification and Regression Trees*
- scikit-learn User Guide: [Decision Trees](https://scikit-learn.org/stable/modules/tree.html)

---



<center>

Thank you!

</center>